# Reconocimiento de Entidades
En este notebook se desarrollará un modelo de Reconocimiento de Entidades (NER) en el contexto de conversaciones del servicio de Atención al Público de la empresa telefónica Celtel. Las conversaciones son entre un operador de la empresa y clientes, que llaman por diversos motivos. En la gran mayoría de casos, el operador les pide datos a los clientes. Esos datos son los que queremos ser capaces de reconocer dentro de los chats.

In [ ]:
# Importar librerias
import pandas as pd
import numpy as np

In [ ]:
# Cargar datos
df = pd.read_csv('dataset.csv')
df.head()

In [ ]:
# Entidades a reconocer
labels = np.delete(df['Tag'].unique(), np.where(df['Tag'].unique() == 'O'))
list(labels)

## Preparación de los Datos

### Lematización

La lematización en el PLN consiste en reducir una palabra a su forma base o lema. El lema representa la forma canónica de una palabra y generalmente corresponde al infinitivo en el caso de los verbos, al singular masculino en el caso de los sustantivos y al grado positivo en el caso de los adjetivos. La lematización es una técnica útil para reducir la variabilidad morfológica de las palabras y establecer relaciones más precisas entre términos similares.

In [ ]:
import spacy
lemmatizer = spacy.load('es_core_news_md')

def lemmatize(sentence):
    doc = lemmatizer(sentence)
    return ' '.join([token.lemma_ for token in doc])

lemmatize('Cliente: Hola, ¿qué tal? Soy María Fernández y me quiero dar de baja de Celtel.')

In [ ]:
# Creo nueva columna con las palabras lematizadas
for i, data in df.groupby('Sentence #'):
    sentence_words_list = data['Word'].values.tolist()
    sentence_words_lens = [len(word) for word in sentence_words_list]
    sentence = ' '.join(sentence_words_list)
    lemmatized_sentence = lemmatize(sentence)
    lemmatized_words_list = lemmatized_sentence.split(' ')
    if (len(lemmatized_words_list) != len(sentence_words_list)):
        lemmatized_words_list = []
        for word in sentence_words_list:
            lemmatized_word = lemmatize(word)
            lemmatized_words_list.append(lemmatized_word.split(' ')[0])
    df.loc[data.index, 'Lemmatized'] = lemmatized_words_list

In [ ]:
print((' ').join(df[df["Sentence #"] == 0]["Word"]))
print((' ').join(df[df["Sentence #"] == 0]["Lemmatized"]))

### Separar datos de entrenamiento y test

In [ ]:
import random
TEST_TRAIN_SPLIT = 0.25
TOTAL_CHATS = max(df['Chat #'])
TEST_CHATS = int(TOTAL_CHATS * TEST_TRAIN_SPLIT)
test_chats_ids = sorted(random.sample(range(2, TOTAL_CHATS), TEST_CHATS))
df_test = df[df['Chat #'].isin(test_chats_ids)]
df_train = df[~df['Chat #'].isin(test_chats_ids)]
print(f"Chats para test: {test_chats_ids}")

### Conversión de .csv a formato spaCy

In [ ]:
def format_dataframe_to_spacy(df):
    res = []
    for i, data in df.groupby('Sentence #'):
        sentence_words_list = data['Word'].values.tolist()
        #sentence_words_list = data['Lemmatized'].values.tolist()
        sentence_words_lens = [len(word) for word in sentence_words_list]
        sentence = ' '.join(sentence_words_list)
        tag_list = data['Tag'].values.tolist()
        start_end_tag = []
        for j, tag in enumerate(tag_list):
            if tag != 'O':
                start = sum(sentence_words_lens[:j]) + j
                end = start + sentence_words_lens[j]
                start_end_tag.append((start, end, tag))
        res.append((sentence, start_end_tag))
    return res

In [ ]:
# Formateo los datos de entrenamiento al formato de Spacy
# Tupla (oración, entidades)
# Entidades: (inicio, fin, tipo)
train_data = format_dataframe_to_spacy(df_train)
train_data[0]

In [ ]:
'Cliente : Hola , ¿ qué tal ? Soy María Fernández y me quiero dar de baja de Celtel .'[33:38]

## Modelo

### Creación

In [ ]:
# Creo el modelo con las entidades
import spacy
nlp = spacy.blank('es')
ner = nlp.add_pipe('ner')
for label in labels:
    ner.add_label(label)

### Entrenamiento

In [ ]:
# Entreno el modelo
from spacy.training import Example
optimizer = nlp.begin_training()
n_iter = 4
for itn in range(n_iter):
    random.shuffle(train_data)
    for raw_text, entity_offsets in train_data:
        doc = nlp.make_doc(raw_text)
        example = Example.from_dict(doc, {"entities": entity_offsets})
        nlp.update([example], sgd=optimizer)

In [ ]:
# # Si se quiere guardar el modelo entrenado
# nlp.to_disk(f'ner_{n_iter}_iterations')

### Resultados

In [ ]:
# Para mostrar los resultados
from spacy import displacy
def find_entities(text):
    return nlp(text)

def print_entities(doc):
    displacy.render(doc, style="ent")

def predict_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        doc = find_entities(text)
        #doc = find_entities(lemmatize(text))
        print_entities(doc)

In [ ]:
# Pruebo con el primer chat de test
predict_file(f'raw_data/chat{str(test_chats_ids[0]).zfill(2)}.txt')

In [ ]:
# Pruebo con el segundo chat de test
predict_file(f'raw_data/chat{str(test_chats_ids[1]).zfill(2)}.txt')

### Métricas

In [ ]:
def get_predicted_df(chat_num):
    predicted = pd.DataFrame(columns=['Chat #', 'Sentence #', 'Word', 'Tag', 'wordspan'])
    first_sentence_num = df_test[df_test['Chat #'] == chat_num]['Sentence #'].unique()[0]
    last_sentence_num = df_test[df_test['Chat #'] == chat_num]['Sentence #'].unique()[-1]
    for i in range(first_sentence_num, last_sentence_num + 1):
        sentence = df_test[df_test['Sentence #'] == i]['Word'].values.tolist()
        sentence = ' '.join(sentence)
        words_lens = [len(word) for word in sentence.split()]
        words_spans = []
        for j, word in enumerate(sentence.split()):
            start = sum(words_lens[:j]) + j
            end = start + words_lens[j]
            words_spans.append((start, end))
        for word, word_span in zip(sentence.split(), words_spans):
            predicted = predicted.append({'Chat #': chat_num, 'Sentence #': i, 'Word': word, 'Tag': 'O', 'wordspan': word_span}, ignore_index=True)
        doc = find_entities(sentence)
        for ent in doc.ents:
            if (ent.start_char, ent.end_char) in words_spans:
                predicted.loc[(predicted['Chat #'] == chat_num) & (predicted['Sentence #'] == i) & (predicted['wordspan'] == (ent.start_char, ent.end_char)), 'Tag'] = ent.label_
    predicted.drop(columns=['wordspan'], inplace=True)
    return predicted

In [ ]:
pred_dfs = []
for id in test_chats_ids:
    pred_dfs.append(get_predicted_df(id))
df_pred = pd.concat(pred_dfs)
df_test.head(20)

In [ ]:
df_test.reset_index(drop=True, inplace=True)
df_pred.reset_index(drop=True, inplace=True)
df_test = df_test[df_test['Tag'] != 'O']
df_pred = df_pred[df_pred.index.isin(df_test[df_test['Tag'] != 'O'].index)]
df_test.head(10)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df_test['Tag'], df_pred['Tag'], zero_division=0))